In [1]:
import pandas as pd
import datasets
import requests
import io

In [3]:
df = pd.read_json("project-4-at-2022-09-26-09-58-1e4e38e4.json")

In [4]:
df = df.drop(axis=1, columns=["Unnamed: 0"])

In [5]:
df.choice.value_counts()

not-illustrated    1430
illustrated         466
Name: choice, dtype: int64

In [6]:
df = df[~df.choice.isna()]

In [7]:
df = df.reset_index(drop=True)

In [8]:
ds = datasets.Dataset.from_pandas(df)

In [9]:
ds

Dataset({
    features: ['image', 'manifest_url', 'license', 'label', 'attribution', 'id', 'choice', 'annotator', 'annotation_id', 'created_at', 'updated_at', 'lead_time'],
    num_rows: 1896
})

In [10]:
ds[0]

{'image': 'https://iiif.archivelab.org/iiif/holylandbiblebok0000cunn$49/full/full/0/default.jpg',
 'manifest_url': 'https://iiif.archivelab.org/iiif/holylandbiblebok0000cunn/manifest.json',
 'license': '',
 'label': '34',
 'attribution': 'The Internet Archive',
 'id': 11967,
 'choice': 'not-illustrated',
 'annotator': 1,
 'annotation_id': 6063,
 'created_at': Timestamp('2022-09-26 09:58:03.514234+0000', tz='UTC'),
 'updated_at': Timestamp('2022-09-26 09:58:03.514258+0000', tz='UTC'),
 'lead_time': 3.618}

In [11]:
from PIL import Image

In [12]:
def load_image(x):
    image_url = x["image"]
    try:
        r = requests.get(image_url, stream=True)
        return {"loaded_image": Image.open(io.BytesIO(r.content))}
    except Exception:
        return {"loaded_image": None}

In [15]:
ds = ds.map(load_image, num_proc=4)

#1:   0%|          | 0/474 [00:00<?, ?ex/s]

#2:   0%|          | 0/474 [00:00<?, ?ex/s]

#3:   0%|          | 0/474 [00:00<?, ?ex/s]

#0:   0%|          | 0/474 [00:00<?, ?ex/s]

In [ ]:
ds.save_to_disk("ia_annotated")

In [ ]:
ds = ds.cast_column("loaded_image", datasets.Image())

In [ ]:
ds.push_to_hub("ImageIN/ImageIn_annotations", embed_external_files=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/Caskroom/miniforge/base/envs/imagein/lib/python3.8/site-packages/huggingface_hub/hf_api.py:2165: FutureWarning: `identical_ok` has no effect and is deprecated. It will be removed in 0.11.0.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]